In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._



val conf=new SparkConf()
      .setMaster("local")//启动本地化计算
      .setAppName("Financial")//设置本程序名称
val sc=new SparkContext(conf)
val spark = SparkSession.builder().appName("Financial_csv").getOrCreate()

conf = org.apache.spark.SparkConf@69741331
sc = org.apache.spark.SparkContext@3b71ef07
spark = org.apache.spark.sql.SparkSession@45da8840


org.apache.spark.sql.SparkSession@45da8840

In [2]:
val f_csv=spark.read.option("header", "true").option("multiLine", true).csv("../data/Financial_Sample.csv")//读取本地文件

f_csv = [Segment: string, Country: string ... 14 more fields]


[Segment: string, Country: string ... 14 more fields]

In [3]:
val fcol = f_csv.columns

")


In [67]:
def col2Float(S: String) = {
    var S_deal = S.trim()
    if (S_deal.charAt(0).equals('$')){
    S_deal = S_deal.substring(1, S_deal.length)
    }
    S_deal = S_deal.trim()
    if (S_deal.indexOf("(") != -1){
        S_deal = S_deal.substring(S_deal.indexOf("(")+1, S_deal.indexOf(")"))
    }
    if (S_deal.charAt(0).equals('-')){
    "0.0"
    }
    else{
        S_deal.replace(",","").toFloat.formatted("%.2f")
    }
    
}
var c2f = udf(col2Float _)

c2f = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


lastException: Throwable = null
col2Float: (S: String)String


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [68]:
col2Float("$(453,375.00)")

453375.00

In [5]:
def columnName_deal(S: String)={
    var S_deal = S.trim
    var a = S_deal.indexOf("(")
    var b = S_deal.indexOf("（")
    if (a != -1){
        if (b<a && b != -1)
        {
            S_deal = S_deal.substring(0,b).trim
        }
        else{
            S_deal = S_deal.substring(0,a).trim
        }  
    }
    else{
        if (b != -1)
        {
            S_deal = S_deal.substring(0,b).trim
        }
        else{
            S_deal = S_deal.trim
        }
    }
    S_deal.replace(" ", "_")
}

columnName_deal: (S: String)String


In [6]:
val str_trim = udf((str: String) => str.trim())

str_trim = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [7]:
def date_deal(S: String)={
    S.trim.replace("/","-")
}
var dd = udf(date_deal _)

dd = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


date_deal: (S: String)String


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [8]:
var newf_csv = f_csv.toDF(fcol.map(columnName_deal(_)):_*)

newf_csv = [Segment: string, Country: string ... 14 more fields]


[Segment: string, Country: string ... 14 more fields]

In [9]:
newf_csv = newf_csv.withColumn("Date", dd(col("Date")))
                   .withColumn("Month_Number", str_trim(col("Month_Number")))
                   .withColumn("Year", str_trim(col("Year")))

newf_csv = [Segment: string, Country: string ... 14 more fields]


[Segment: string, Country: string ... 14 more fields]

In [10]:
val newfcol = newf_csv.columns

newfcol = Array(Segment, Country, Product, Discount_Band, Units_Sold, Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit, Date, Month_Number, Month_Name, Year)


Array(Segment, Country, Product, Discount_Band, Units_Sold, Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit, Date, Month_Number, Month_Name, Year)

In [11]:
newfcol.map(column =>{
    newf_csv = newf_csv.withColumn(column, str_trim(col(column)))
})

Array((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ())

In [12]:
val float_col = newfcol.slice(5,12)

float_col = Array(Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit)


Array(Manufacturing_Price, Sale_Price, Gross_Sales, Discounts, Sales, COGS, Profit)

In [28]:
float_col.toSeq.foldLeft("")(_ + "," + _).substring(1)

Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit

In [ ]:
float_col.map(column =>{
    newf_csv = newf_csv.withColumn(column, c2f(col(column)))
})

In [16]:
newf_csv.select("Gross_Sales").collect()

Array([$32,370.00], [$26,420.00], [$32,670.00], [$13,320.00], [$37,050.00], [$529,550.00], [$13,815.00], [$30,216.00], [$37,980.00], [$18,540.00], [$37,050.00], [$333,187.50], [$287,400.00], [$15,022.00], [$43,125.00], [$9,225.00], [$5,840.00], [$14,610.00], [$30,216.00], [$352,100.00], [$4,404.00], [$6,181.00], [$8,235.00], [$236,400.00], [$37,080.00], [$8,001.00], [$603,750.00], [$10,944.00], [$32,280.00], [$36,340.00], [$529,550.00], [$10,451.00], [$225,500.00], [$25,932.00], [$352,100.00], [$18,540.00], [$352,625.00], [$43,125.00], [$600,300.00], [$34,056.00], [$32,670.00], [$13,320.00], [$534,450.00], [$645,300.00], [$36,340.00], [$962,500.00], [$23,436.00], [$527,437.50], [$37,980.00], [$11,802.00], [$25,692.00], [$8,001.00], [$9,225.00], [...

In [18]:
import org.apache.spark.sql.SaveMode

In [19]:
newf_csv.write.mode(SaveMode.Append).partitionBy("Year", "Month_Number", "Date").saveAsTable("financial")

In [60]:
var pre_fcsv = spark.sql("select Country, Product, Units_Sold, Sales, COGS, Profit, Year, Month_Number as Month from financial ")

pre_fcsv = [Country: string, Product: string ... 6 more fields]


lastException: Throwable = null


[Country: string, Product: string ... 6 more fields]

In [40]:
pre_fcsv.show

+--------------------+---------+----------+-----------+-----------+-----------+----+-----+
|             Country|  Product|Units_Sold|      Sales|       COGS|     Profit|Year|Month|
+--------------------+---------+----------+-----------+-----------+-----------+----+-----+
|United States of ...|    Paseo|      1143|  $8,001.00|  $5,715.00|  $2,286.00|2014|   10|
|United States of ...| Amarilla|      1143|  $8,001.00|  $5,715.00|  $2,286.00|2014|   10|
|              Mexico|Carretera|      1397|$484,060.50|$363,220.00|$120,840.50|2014|   10|
|              Mexico|      VTT|      1397|$484,060.50|$363,220.00|$120,840.50|2014|   10|
|              Canada|Carretera|      1295| $15,229.20|  $3,885.00| $11,344.20|2014|   10|
|United States of ...|  Montana|      1566| $30,693.60| $15,660.00| $15,033.60|2014|   10|
|              Canada|    Paseo|      1295| $15,229.20|  $3,885.00| $11,344.20|2014|   10|
|United States of ...|     Velo|      1566| $30,693.60| $15,660.00| $15,033.60|2014|   10|

In [61]:
pre_fcsv.select("Sales").show

+-----------+
|      Sales|
+-----------+
|  $8,001.00|
|  $8,001.00|
|$484,060.50|
|$484,060.50|
| $15,229.20|
| $30,693.60|
| $15,229.20|
| $30,693.60|
|$986,811.00|
|$986,811.00|
|$243,591.25|
|$243,591.25|
| $29,246.40|
| $29,246.40|
|$582,048.00|
|$582,048.00|
|$731,472.00|
|$731,472.00|
| $17,881.85|
| $17,881.85|
+-----------+
only showing top 20 rows



In [62]:
pre_fcsv = pre_fcsv.withColumn("Sales", c2f(col("Sales")))

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [63]:
pre_fcsv = pre_fcsv.withColumn("COGS", c2f(col("COGS")))
pre_fcsv.show

+--------------------+---------+----------+---------+---------+-----------+----+-----+
|             Country|  Product|Units_Sold|    Sales|     COGS|     Profit|Year|Month|
+--------------------+---------+----------+---------+---------+-----------+----+-----+
|United States of ...|    Paseo|      1143|  8001.00|  5715.00|  $2,286.00|2014|   10|
|United States of ...| Amarilla|      1143|  8001.00|  5715.00|  $2,286.00|2014|   10|
|              Mexico|Carretera|      1397|484060.50|363220.00|$120,840.50|2014|   10|
|              Mexico|      VTT|      1397|484060.50|363220.00|$120,840.50|2014|   10|
|              Canada|Carretera|      1295| 15229.20|  3885.00| $11,344.20|2014|   10|
|United States of ...|  Montana|      1566| 30693.60| 15660.00| $15,033.60|2014|   10|
|              Canada|    Paseo|      1295| 15229.20|  3885.00| $11,344.20|2014|   10|
|United States of ...|     Velo|      1566| 30693.60| 15660.00| $15,033.60|2014|   10|
|             Germany|     Velo|      2877|

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [69]:
pre_fcsv = pre_fcsv.withColumn("Profit", c2f(col("Profit")))
pre_fcsv.show

+--------------------+---------+----------+---------+---------+---------+----+-----+
|             Country|  Product|Units_Sold|    Sales|     COGS|   Profit|Year|Month|
+--------------------+---------+----------+---------+---------+---------+----+-----+
|United States of ...|    Paseo|      1143|  8001.00|  5715.00|  2286.00|2014|   10|
|United States of ...| Amarilla|      1143|  8001.00|  5715.00|  2286.00|2014|   10|
|              Mexico|Carretera|      1397|484060.50|363220.00|120840.50|2014|   10|
|              Mexico|      VTT|      1397|484060.50|363220.00|120840.50|2014|   10|
|              Canada|Carretera|      1295| 15229.20|  3885.00| 11344.20|2014|   10|
|United States of ...|  Montana|      1566| 30693.60| 15660.00| 15033.60|2014|   10|
|              Canada|    Paseo|      1295| 15229.20|  3885.00| 11344.20|2014|   10|
|United States of ...|     Velo|      1566| 30693.60| 15660.00| 15033.60|2014|   10|
|             Germany|     Velo|      2877|986811.00|748020.00|23

pre_fcsv = [Country: string, Product: string ... 6 more fields]


[Country: string, Product: string ... 6 more fields]

In [93]:
pre_fcsv = pre_fcsv.withColumn("YM", concat_ws("-",col("Year"), col("Month"))).drop("Year","Month")

pre_fcsv = [Country: string, Product: string ... 5 more fields]


[Country: string, Product: string ... 5 more fields]

In [76]:
val float_2f = udf((str: String) => str.toFloat.formatted("%.2f"))

float_2f = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

# 月度销售额top3国家

In [97]:
var top3_sales_country = 
pre_fcsv.groupBy("Country", "YM")
        .agg(sum("Sales") as "Sales_c")
        .orderBy(- col("Sales_c"))
        .withColumn("Sales_c", float_2f(col("Sales_c")))
top3_sales_country.take(3)

top3_sales_country = [Country: string, YM: string ... 1 more field]


Array([Canada,2014-12,3959729.50], [Germany,2014-10,3421587.25], [France,2014-10,3379661.50])

# 月度利润率top3国家

In [101]:
var top3_profitraio_country = 
pre_fcsv.groupBy("Country", "YM")
        .agg(sum("COGS") as "COGS_c", sum("Profit") as "Profit_c")
        .withColumn("Profit_ratio", col("Profit_c")/col("COGS_c"))
        .orderBy(- col("Profit_ratio"))
        .withColumn("Profit_ratio", float_2f(col("Profit_ratio")))
top3_profitraio_country.select("Country", "YM", "Profit_ratio").take(3)

top3_profitraio_country = [Country: string, YM: string ... 3 more fields]


Array([France,2013-9,0.31], [Germany,2013-10,0.30], [Canada,2013-10,0.28])

# 月度畅销top2商品

In [106]:
var top2_sold_product = 
pre_fcsv.groupBy("Product", "YM")
        .agg(sum("Units_Sold") as "Units_Sold_p")
        .orderBy(- col("Units_Sold_p"))
top2_sold_product.take(2)

top2_sold_product = [Product: string, YM: string ... 1 more field]


Array([Paseo,2014-12,33763.0], [Paseo,2014-10,29623.0])

# 月度利润额最低top2商品

In [109]:
var top2_sales_loe_product = 
pre_fcsv.groupBy("Product", "YM")
        .agg(sum("Profit") as "Profit_p")
        .orderBy(col("Profit_p"))
        .withColumn("Profit_p", float_2f(col("Profit_p")))
top2_sales_loe_product.take(2)

top2_sales_loe_product = [Product: string, YM: string ... 1 more field]


Array([Montana,2014-2,7220.82], [VTT,2014-3,9699.56])